In [64]:
import numpy as np
import pandas as pd
import math
from datetime import datetime

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import pickle
import math
import optuna
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn import preprocessing
from NFLUtils import NFLUtils
nfl_utils = NFLUtils()
from MLBUtils import MLBUtils
mlb_utils = MLBUtils()


import matplotlib.pyplot as plt
import numpy as np
import torch
from datetime import datetime, timedelta
import math
import torch.nn as nn
import requests as r
from bs4 import BeautifulSoup as bs
import umap
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows
pd.set_option('display.max_colwidth', None) # Avoid truncating individual column values

## Global variables

[{'Season': 2024, 'Home_Team': 'PHI', 'Visitor_Team': 'WAS', 'Date': '2025-01-26'}, {'Season': 2024, 'Home_Team': 'KAN', 'Visitor_Team': 'BUF', 'Date': '2025-01-26'}]


In [65]:
current_season = 2025

In [66]:
def getAlternateAbbrv(teamName, isoDate, df):
    """
    This will return either 'None' (to signify & ignore double header) or the team name that matches 
    """
    alternateAbbrv = [
        ['CUB', 'CHN', 'CHC'],
        ['NYM', 'NYN'],
        ['STL', 'SLN'],
        ['SDG', 'SDN', 'SD'],
        ['KAN', 'KCA'],
        ['LAA', 'ANA'],
        ['LAD', 'LAN', 'LOS', 'LA'],
        ['WAS', 'WSH'],
        ['SFO', 'SFN', 'SF'],
        ['CWS', 'CHA'],
        ['TAM', 'TBA', 'TB'],
        ['NYY', 'NYA'],
        ['CHW', 'CHA'],
        ['KC', 'KCA'],
        ['OAK', 'ATH']
    ]
    isoDateTime = pd.to_datetime(isoDate).date()
    
    # Reduce to 1 dimension, check all alternate abbreviations
    alternateAbbrv = [item for subList in alternateAbbrv for item in subList if teamName in subList]
    alternateAbbrv.append(teamName)
    for teamAbbrv in alternateAbbrv:
        filtered_df = df[((df['Home_Team'] == teamAbbrv) | (df['Visitor_Team'] == teamAbbrv))]
        if len(filtered_df) > 0:
            return teamAbbrv
    print(f"WARNING: Cannot find team abbrv for {teamName}")
    return None

In [67]:
# Import newCombined
df = pd.read_csv("./baseballData/NewCombined.csv", index_col=False, low_memory=False)
print(f"Before season filter: {df.shape}")
df.columns = df.columns.str.replace(' ', '_')
# Convert 'date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format="%Y-%m-%d")

# Filter for rows where the year is current_season
df = df[df['Date'].dt.year == current_season]
print(f"After season filter: {df.shape}")
df.head()

Before season filter: (33321, 129)
After season filter: (469, 129)


,Date,Home_Team,Home_Wins,Home_Losses,Visitor_Team,Visitor_Wins,Visitor_Losses,Final_Score,Visitor_Final_Score,Home_Final_Score,Winning_Team,Home/Victory_Win,Game_Time,Winning_Pitcher,Visiting_Pitcher,Winning_Pitcher_Team,Losing_Pitcher_Team,Game_Time_(Duration),Attendance,Field_Conditions,Temperature,Wind_Direction,Cloud_Coverage,HP_Umpire,1B_Umpire,2B_Umpire,3B_Umpire,LF_Umpire,RF_Umpire,V_AB,V_R,V_H,V_RBI,V_BB,V_SO,V_PA,V_BA,V_OBP,V_SLG,V_OPS,V_PIT,V_STR,V_WPA,V_aLI,V_WPA+,V_WPA-,V_cWPA,V_acLI,V_RE24,V_PO,V_A,V_Batting_Details,H_AB,H_R,H_H,H_RBI,H_BB,H_SO,H_PA,H_BA,H_OBP,H_SLG,H_OPS,H_PIT,H_STR,H_WPA,H_aLI,H_WPA+,H_WPA-,H_cWPA,H_acLI,H_RE24,H_PO,H_A,H_Batting_Details,V_IP,V_P_H,V_P_R,V_ER,V_P_BB,V_P_SO,V_HR,V_ERA,V_BF,V_PIT.1,V_STR.1,V_CTCT,V_STS,V_STL,V_GB,V_FB,V_LD,V_UNK,V_GSC,V_IR,V_IS,V_P_WPA,V_P_aLI,V_P_cWPA,V_P_acLI,V_P_RE24,H_IP,H_P_H,H_P_R,H_ER,H_P_BB,H_P_SO,H_HR,H_ERA,H_BF,H_PIT.1,H_STR.1,H_CTCT,H_STS,H_STL,H_GB,H_FB,H_LD,H_UNK,H_GSC,H_IR,H_IS,H_P_WPA,H_P_aLI,H_P_cWPA,H_P_acLI,H_P_RE24,H_start_odds,V_start_odds
32852,2025-05-01,SFN,19,13,COL,6,25,4-3,4,3,COL,V,6:45 p.m.,Angel Chivilli,Tyler Rogers,COL,SFN,2:19,"27,198",Unknown,54° F,14mph out to Centerfield,Cloudy,David Rackley,Edwin Moscoso,Tom Hanahan,Chris Guccione,NaN,NaN,34,4,8,4,2,6,36,0.235,0.278,0.382,0.660,126,79,0.233,1.20,1.015,-0.784,0.01%,0.10,-0.2,27,11,-,34,3,9,3,2,5,36,0.265,0.306,0.412,0.717,121,84,-0.266,1.35,0.673,-0.938,-0.18%,1.49,-1.2,27,11,-,9.0,9,3,3,2,5,1,3.00,36,121,84,53,7,24,12,17,7,0,49,1,0,0.268,1.35,0.01%,0.11,1.2,9.0,8,4,4,2,6,1,4.0,36,126,79,48,13,18,13,15,4,0,58,0,0,-0.233,1.20,-0.15%,1.32,0.2,3.20,1.363636
32853,2025-05-01,NYN,21,11,ARI,17,14,4-2,4,2,ARI,V,1:10 p.m.,Zac Gallen,Kodai Senga,ARI,NYN,2:55,"36,239",Unknown,66° F,8mph from Right to Left,Cloudy,John Bacon,CB Bucknor,Adam Beck,Scott Barry,NaN,NaN,34,4,9,4,5,11,41,0.265,0.350,0.471,0.821,174,104,0.108,1.05,0.814,-0.705,0.06%,1.02,-0.3,27,9,-,32,2,5,2,4,13,36,0.156,0.250,0.375,0.625,145,91,-0.392,1.26,0.646,-1.039,-0.29%,1.55,-2.3,27,6,-,9.0,5,2,2,4,13,2,2.00,36,145,91,45,14,32,10,9,4,0,69,0,0,0.393,1.26,0.23%,1.22,2.3,9.0,9,4,4,5,11,2,4.0,41,174,104,53,21,29,7,17,2,0,51,2,1,-0.108,1.05,-0.08%,1.29,0.3,2.40,1.606061
32854,2025-05-01,CLE,18,13,MIN,13,19,3-4,3,4,CLE,H,1:10 p.m.,Kolby Allard,Justin Topa,CLE,MIN,3:14,"15,968",Unknown,78° F,15mph out to Leftfield,Cloudy,Andy Fletcher,Jansen Visconti,Ben May,Malachi Moore,NaN,NaN,40,3,13,3,3,4,44,0.325,0.364,0.400,0.764,192,124,-0.286,1.99,1.229,-1.514,-0.14%,1.58,-2.5,28,10,-,32,4,6,4,8,11,41,0.188,0.350,0.281,0.631,178,110,0.214,1.76,1.185,-0.969,0.15%,2.13,-0.6,30,7,-,9.1,6,4,3,8,11,1,2.89,41,178,110,61,23,26,15,7,1,0,51,2,0,-0.215,1.76,-0.10%,1.40,0.6,10.0,13,3,2,3,4,0,1.8,44,192,125,77,16,32,10,27,10,0,61,3,1,0.286,1.99,0.20%,2.40,2.5,2.10,1.769231
32855,2025-05-01,CIN,17,15,SLN,14,18,1-9,1,9,CIN,H,12:40 p.m.,Graham Ashcraft,Matthew Liberatore,CIN,SLN,2:21,"15,513",Unknown,76° F,15mph from Right to Left,Cloudy,Nate Tomlinson,Mark Wegner,Bruce Dreckman,Shane Livensparger,NaN,NaN,29,1,3,0,4,9,33,0.103,0.212,0.103,0.316,132,89,-0.261,0.72,0.252,-0.513,-0.12%,0.58,-3.8,24,8,-,33,9,9,9,4,7,39,0.273,0.359,0.455,0.814,144,89,0.240,0.51,0.499,-0.258,0.14%,0.51,4.7,27,5,-,8.0,9,9,8,4,7,2,9.00,39,144,89,47,10,32,13,14,7,0,48,2,2,-0.239,0.51,-0.11%,0.41,-4.7,9.0,3,1,0,4,9,0,0.0,33,132,89,50,14,25,5,15,4,0,53,0,0,0.261,0.72,0.16%,0.72,3.8,2.00,1.833333
32856,2025-05-01,PIT,12,20,CHN,19,13,8-3,8,3,CHN,V,12:35 p.m.,Colin Rea,Paul Skenes,CHN,PIT,2:32,"13,633",Unknown,81° F,12mph out to Centerfield,Cloudy,Lance Barrett,Carlos Torres,Charlie Ramos,Alfonso MÃ¡rquez,NaN,NaN,37,8,11,8,4,2,41,0.297,0.366,0.649,1.015,150,96,0.300,0.84,0.963,-0.665,0.21%,1.00,3.4,27,13,-,32,3,5,3,3,2,35,0.156,0.229,0.281,0.510,123,80,-0.203,1.10,0.512,-0.715,-0.07%,0.62,-1.6,27,8,-,9.0,5,3,2,3,2,1,2.00,35,123,80,47,5,28,12,18,5,0,56,0,0,0.201,1.10,0.14%,1.30,1.6,9.0,11,8,8,4,2,4,8.0,41,150,96,59,15,22,11,24,7,0,43,1,1,-0.299,0.84,-0.10%,0.48,-3.4,2.05,1.800000


In [68]:
# Retrieve games for current date
day_adjust_back = 3
iso_date_pre = (datetime.now() - timedelta(days=day_adjust_back)).replace(hour=0, minute=0, second=0, microsecond=0)
iso_date = iso_date_pre.strftime("%Y-%m-%d")
current_date_games = nfl_utils.getMLBOddsSharkData(iso_date)
current_date_games_filtered = []
print(iso_date_pre)
print(df.shape)
df = df[df['Date'] < iso_date_pre]
print(df.shape)
for game in current_date_games:
    # Home_Team & Visitor_Team must match
    game["Home_Team"] = getAlternateAbbrv(game["Home_Team"], game["Date"], df)
    game["Visitor_Team"] = getAlternateAbbrv(game["Visitor_Team"], game["Date"], df)
    if game["Home_Team"] is None or game["Visitor_Team"] is None:
        continue
    current_date_games_filtered.append(game)
    
    game["H_start_odds"] = mlb_utils.americanToDecimal(game["Home_Odds"])
    game["V_start_odds"] = mlb_utils.americanToDecimal(game["Visitor_Odds"])
    del game["Home_Odds"]
    del game["Visitor_Odds"]
    print(game)
# current_date_games = pd.DataFrame.from_records(current_date_games)

https://www.oddsshark.com/api/scores/mlb/2025-05-01?_format=json
2025-05-01 00:00:00
(469, 129)
(443, 129)
{'Date': '2025-05-01', 'Rot': 952, 'Home_Team': 'PIT', 'Visitor_Team': 'CHN', 'H_start_odds': 1.8, 'V_start_odds': 2.05}
{'Date': '2025-05-01', 'Rot': 954, 'Home_Team': 'CIN', 'Visitor_Team': 'SLN', 'H_start_odds': 2.0, 'V_start_odds': 1.8333333333333335}
{'Date': '2025-05-01', 'Rot': 962, 'Home_Team': 'CLE', 'Visitor_Team': 'MIN', 'H_start_odds': 2.1, 'V_start_odds': 1.7692307692307692}
{'Date': '2025-05-01', 'Rot': 964, 'Home_Team': 'TBA', 'Visitor_Team': 'KCA', 'H_start_odds': 1.625, 'V_start_odds': 2.35}
{'Date': '2025-05-01', 'Rot': 956, 'Home_Team': 'NYN', 'Visitor_Team': 'ARI', 'H_start_odds': 1.606060606060606, 'V_start_odds': 2.4}
{'Date': '2025-05-01', 'Rot': 972, 'Home_Team': 'CHA', 'Visitor_Team': 'MIL', 'H_start_odds': 2.4, 'V_start_odds': 1.6451612903225805}
{'Date': '2025-05-01', 'Rot': 966, 'Home_Team': 'TEX', 'Visitor_Team': 'ATH', 'H_start_odds': 1.76923076923076

### Define cont_cols, y_col (taken from MLB_ANN)

In [69]:

# df["Home_Team"] = df["Home_Team"].astype(str) 

current_date_games = pd.DataFrame(current_date_games, columns=df.columns)
current_date_games['Date'] = pd.to_datetime(current_date_games['Date'], format="%Y-%m-%d")
# current_date_games = current_date_games.reindex(columns=df.columns)  # Only keep relevant columns
# print(current_date_games["Home_Team"].dtype)
# current_date_games.head()
df = pd.concat([df, current_date_games], ignore_index=True)



# Replace spaces with underscores in column names
df.columns = df.columns.str.replace(' ', '_')

# Limit to first n rows
# df = df[:2000]
# Sort by date from least to greatest
df = df.sort_values(by='Date')

# Create the H_Won column
df['H_Won'] = np.where(df['Home_Final_Score'] > df['Visitor_Final_Score'], 1.0, 0.0)

# Create the H_final_allowed, V_final_allowed columns
df['H_Final_Allowed'] = df['Visitor_Final_Score']
df['V_Final_Allowed'] = df['Home_Final_Score']

df['H_Final_Score'] = df['Home_Final_Score']
df['V_Final_Score'] = df['Visitor_Final_Score']

df = df.sort_values(by='Date')



track_cols = [
    # General
    'Date',  # Date

    # Random
    'AB',
    'R',
    'H',
    'BB',
    'PA',
    
    
    # Batting stats
    'SO',    # Strikeouts
    'BA',    # Batting Average
    'OBP',   # On-Base Percentage
    'SLG',   # Slugging Percentage
    'OPS',   # On-Base Plus Slugging
    'RBI',   # Runs Batted In
    'PIT',   # Pitches
    'STR',   # Strikes
    'WPA',   # Win Probability Added
    'aLI',   # Average Leverage Index

    # Pitching stats
    'P_H',   # Hits Allowed by Pitcher
    'P_R',   # Runs Allowed by Pitcher
    'ER',    # Earned Runs by Pitcher
    'P_BB',  # Walks by Pitcher
    'P_SO',  # Strikeouts by Pitcher
    'HR',    # Home Runs Allowed by Pitcher
    'ERA',   # Earned Run Average
    'CTCT',  # Contact Rate
    'STS',   # Swings and Misses (Strikes)
    'STL',   # Stolen Bases Against
    'GB',    # Ground Balls
    'FB',    # Fly Balls
    'LD',    # Line Drives
    'GSC',   # Game Score
    'P_WPA', # Pitcher Win Probability Added
    'P_aLI',  # Pitcher Average Leverage Index

    # Odds 
    'start_odds',

    # Remove later
    'Final_Score',
    'Final_Allowed'
]



In [70]:
df.tail(15)
# print(df["Home_Team"].dtype)

,Date,Home_Team,Home_Wins,Home_Losses,Visitor_Team,Visitor_Wins,Visitor_Losses,Final_Score,Visitor_Final_Score,Home_Final_Score,Winning_Team,Home/Victory_Win,Game_Time,Winning_Pitcher,Visiting_Pitcher,Winning_Pitcher_Team,Losing_Pitcher_Team,Game_Time_(Duration),Attendance,Field_Conditions,Temperature,Wind_Direction,Cloud_Coverage,HP_Umpire,1B_Umpire,2B_Umpire,3B_Umpire,LF_Umpire,RF_Umpire,V_AB,V_R,V_H,V_RBI,V_BB,V_SO,V_PA,V_BA,V_OBP,V_SLG,V_OPS,V_PIT,V_STR,V_WPA,V_aLI,V_WPA+,V_WPA-,V_cWPA,V_acLI,V_RE24,V_PO,V_A,V_Batting_Details,H_AB,H_R,H_H,H_RBI,H_BB,H_SO,H_PA,H_BA,H_OBP,H_SLG,H_OPS,H_PIT,H_STR,H_WPA,H_aLI,H_WPA+,H_WPA-,H_cWPA,H_acLI,H_RE24,H_PO,H_A,H_Batting_Details,V_IP,V_P_H,V_P_R,V_ER,V_P_BB,V_P_SO,V_HR,V_ERA,V_BF,V_PIT.1,V_STR.1,V_CTCT,V_STS,V_STL,V_GB,V_FB,V_LD,V_UNK,V_GSC,V_IR,V_IS,V_P_WPA,V_P_aLI,V_P_cWPA,V_P_acLI,V_P_RE24,H_IP,H_P_H,H_P_R,H_ER,H_P_BB,H_P_SO,H_HR,H_ERA,H_BF,H_PIT.1,H_STR.1,H_CTCT,H_STS,H_STL,H_GB,H_FB,H_LD,H_UNK,H_GSC,H_IR,H_IS,H_P_WPA,H_P_aLI,H_P_cWPA,H_P_acLI,H_P_RE24,H_start_odds,V_start_odds,H_Won,H_Final_Allowed,V_Final_Allowed,H_Final_Score,V_Final_Score
272,2025-04-30,SEA,18.0,12.0,ANA,12.0,17.0,3-9,3.0,9.0,SEA,H,1:10 p.m.,Casey Legumina,Reid Detmers,SEA,ANA,2:41,"16,229",Unknown,56° F,6mph from Left to Right,Cloudy,Chris Segal,Alex MacKay,Tripp Gibson,Jim Wolf,NaN,NaN,36.0,3.0,10.0,3.0,3.0,4.0,39.0,0.278,0.333,0.361,0.694,139.0,90.0,-0.053,1.23,0.762,-0.815,-0.03%,1.08,-1.0,24,8.0,-,36.0,9.0,12.0,8.0,3.0,6.0,39.0,0.333,0.385,0.444,0.829,147.0,99.0,0.446,0.98,0.953,-0.505,0.34%,1.22,5.4,27,10.0,-,8.0,12.0,9.0,8.0,3.0,6.0,1.0,9.00,39.0,147.0,99.0,60.0,17.0,22.0,14.0,16.0,4.0,0.0,57.0,2,1,-0.447,0.98,-0.23%,0.85,-5.4,9.0,10.0,3.0,3.0,3.0,4.0,1.0,3.0,39.0,139.0,92.0,57.0,9.0,26.0,12.0,20.0,8.0,0.0,44.0,0,0,0.053,1.23,0.04%,1.54,1.0,1.714286,2.200000,1.0,3.0,9.0,9.0,3.0
260,2025-04-30,TEX,16.0,15.0,ATH,16.0,15.0,7-1,7.0,1.0,ATH,V,7:05 p.m.,Grant Holman,Robert Garcia,ATH,TEX,2:51,"19,803",NaN,74° F,0mph,In Dome,James Hoye,D.J. Reyburn,John Libka,Sean Barber,NaN,NaN,36.0,7.0,11.0,7.0,2.0,10.0,38.0,0.306,0.342,0.500,0.842,146.0,92.0,0.093,1.15,0.881,-0.790,0.06%,1.22,2.7,27,7.0,-,35.0,1.0,9.0,1.0,3.0,9.0,39.0,0.257,0.316,0.257,0.573,159.0,106.0,-0.406,1.76,0.766,-1.175,-0.27%,1.98,-3.3,27,11.0,-,9.0,9.0,1.0,1.0,3.0,9.0,0.0,1.00,39.0,159.0,106.0,72.0,13.0,21.0,12.0,15.0,9.0,0.0,50.0,2,0,0.408,1.76,0.26%,1.86,3.3,9.0,11.0,7.0,7.0,2.0,10.0,2.0,7.0,38.0,146.0,92.0,49.0,19.0,24.0,11.0,15.0,8.0,0.0,64.0,4,4,-0.093,1.15,-0.06%,1.29,-2.7,2.350000,1.625000,0.0,7.0,1.0,1.0,7.0
269,2025-04-30,SDN,19.0,11.0,SFN,19.0,12.0,3-5,3.0,5.0,SDN,H,1:10 p.m.,Michael King,Landen Roupp,SDN,SFN,2:34,"37,698",Unknown,65° F,10mph from Left to Right,Cloudy,Phil Cuzzi,Tony Randazzo,Jacob Metz,Dan Bellino,NaN,NaN,31.0,3.0,5.0,3.0,3.0,12.0,35.0,0.161,0.257,0.355,0.612,150.0,94.0,-0.353,0.98,0.358,-0.708,-0.28%,1.28,-1.6,24,9.0,-,32.0,5.0,10.0,5.0,4.0,6.0,36.0,0.312,0.389,0.469,0.858,143.0,88.0,0.149,0.68,0.535,-0.385,0.12%,0.87,1.0,27,5.0,-,8.0,10.0,5.0,5.0,4.0,6.0,1.0,5.62,36.0,143.0,88.0,50.0,11.0,27.0,15.0,11.0,4.0,0.0,34.0,3,1,-0.149,0.68,-0.12%,0.89,-1.0,9.0,5.0,3.0,3.0,3.0,12.0,2.0,3.0,35.0,150.0,95.0,49.0,17.0,29.0,3.0,16.0,3.0,0.0,64.0,2,0,0.350,0.98,0.27%,1.25,1.6,1.714286,2.200000,1.0,3.0,5.0,5.0,3.0
267,2025-04-30,CLE,17.0,13.0,MIN,13.0,18.0,2-4,2.0,4.0,CLE,H,6:10 p.m.,Cade Smith,Pablo LÃ³pez,CLE,MIN,2:24,"15,523",Unknown,49° F,9mph in from Centerfield,Cloudy,Malachi Moore,Andy Fletcher,Jansen Visconti,Ben May,NaN,NaN,32.0,2.0,6.0,1.0,2.0,13.0,34.0,0.188,0.235,0.281,0.517,139.0,90.0,-0.479,0.97,0.250,-0.728,-0.25%,0.83,-2.4,24,9.0,-,29.0,4.0,5.0,3.0,1.0,10.0,30.0,0.172,0.200,0.345,0.545,127.0,82.0,0.022,1.02,0.605,-0.585,0.01%,1.21,0.1,27,6.0,-,8.0,5.0,4.0,4.0,1.0,10.0,1.0,4.50,30.0,127.0,82.0,48.0,14.0,20.0,11.0,8.0,2.0,0.0,63.0,1,1,-0.022,1.02,-0.01%,0.88,-0.1,9.0,6.0,2.0,2.0,2.0,13.0,1.0,2.0,34.0,139.0,90.0,49.0,22.0,19.0,10.0,9.0,3.0,0.0,70.0,1,0,0.479,0.97,0.34%,1.15,2.4,1.952381,1.869565,1.0,2.0,4.0,4.0,2.0
450,2

### 2. Get track dict, format to EMA

In [71]:
# Get track_dict
track_dict = {}

for row in df.itertuples():
    year = row.Date.year
    # year = row.Season
    home_team = row.Home_Team # Home Team
    visitor_team = row.Visitor_Team # Visitor team

    
    # Home or visitor team has < minimum_window total games
    for col in track_cols:
        home_column_name = f'{year}_{home_team}_{col}'
        visitor_column_name = f'{year}_{visitor_team}_{col}'
        
        # Home team
        home_col = col if col == 'Date' else 'H_' + col
        if home_column_name in track_dict:
            track_dict[home_column_name].append(getattr(row, home_col))
        else:
            track_dict[home_column_name] = [getattr(row, home_col)]
        
        # Visitor team
        visitor_col = col if col == 'Date' else 'V_' + col
        if visitor_column_name in track_dict:
            track_dict[visitor_column_name].append(getattr(row, visitor_col))
        else:
            track_dict[visitor_column_name] = [getattr(row, visitor_col)]


minimum_window = 4
ema_span = 7
print(df.shape)

show_examples = False

indices_to_drop = []
current_count = 0

for row in df.itertuples():
    if current_count % 300 == 0:
        print(f'{current_count}/{df.shape[0]}')
    current_count = current_count + 1
    index = row.Index
    year = row.Date.year
    # year = row.Season
    home_team = row.Home_Team
    visitor_team = row.Visitor_Team
    # Home team min window
    home_date_column = f'{year}_{home_team}_Date'
    visitor_date_column = f'{year}_{visitor_team}_Date'

    # Current row is older than Home team at min_window
    if row.Date <= track_dict[home_date_column][minimum_window]:
        indices_to_drop.append(index)
        continue
    # Current row is older than Visitor team at min_window
    if row.Date <= track_dict[visitor_date_column][minimum_window]:
        indices_to_drop.append(index)
        continue

    home_date_index = track_dict[home_date_column].index(row.Date)
    visitor_date_index = track_dict[visitor_date_column].index(row.Date)
    # print(f'H: {home_date_index} V: {visitor_date_index}')

    # print("----- Track Cols -----")
    # Update df to have average for each track_cols (Ignoring 'Date', the 1-2nd item)
    for col in track_cols:
        if col in ['Date', 'start_odds', 'halftime_odds', 'Final_Score', 'Final_Allowed']:
            continue
        else:
            # print(col)
            # Convert to float
            if df.dtypes['H_' + col] == 'int64' or df.dtypes['V_' + col] == 'int64':
                df = df.astype({f'H_{col}': 'float64', f'V_{col}': 'float64'})
            # Update df to have average for each track_cols (Ignoring 'Date', .. items)
            # Update home
            home_col_list = track_dict[f'{year}_{home_team}_{col}'][:home_date_index-1]
            dataframe_val = pd.DataFrame({'value': home_col_list})
            ema = dataframe_val['value'].ewm(span=min(ema_span, len(home_col_list)), adjust=True).mean().iloc[-1]
            df.at[index, 'H_' + col] = ema

            # Update Visitor
            visitor_col_list = track_dict[f'{year}_{visitor_team}_{col}'][:visitor_date_index-1]
            dataframe_val = pd.DataFrame({'value': visitor_col_list})
            ema = dataframe_val['value'].ewm(span=min(ema_span, len(visitor_col_list)), adjust=True).mean().iloc[-1]
            df.at[index, 'V_' + col] = ema


    # --------------------------------------------------- 
    # ------------------ Custom Columns -----------------
    # --------------------------------------------------- 
    #
    
    # 1. Add variant of Bill James pythagorean expectation (NFL).
    # Recent games weighted more heavily since 'Final' columns not excluded from the above loop
    home_points_for = sum(track_dict[f'{year}_{home_team}_Final_Score'][:home_date_index-1])
    home_points_against = sum(track_dict[f'{year}_{home_team}_Final_Allowed'][:home_date_index-1])
    if math.isnan(home_points_for) or math.isnan(home_points_against):
        print(row.Date)
        print(home_points_for)
        print(home_points_against)
    df.at[index, 'H_pythagorean'] = home_points_for**1.83 / (home_points_for**1.83 + home_points_against**1.83)

    visitor_points_for = sum(track_dict[f'{year}_{visitor_team}_Final_Score'][:visitor_date_index-1])
    visitor_points_against = sum(track_dict[f'{year}_{visitor_team}_Final_Allowed'][:visitor_date_index-1])
    df.at[index, 'V_pythagorean'] = visitor_points_for**1.83 / (visitor_points_for**1.83 + visitor_points_against**1.83)

df.drop(indices_to_drop, inplace=True)

# Add custom metrics to track_cols so it creates the difference (D_) column
track_cols.append('pythagorean')
for col in track_cols[1:]:
    # cont_cols.append('D_' + col) This is unnecessary? 5/3/25
    df['D_' + col] = (df['H_' + col] - df['V_' + col]).round(3) # Round to 3 sig figs

# Remove columns that begin with H_ or V_ in df
y_col = ['H_Won', 'H_start_odds', 'V_start_odds']
df = df.loc[:, ~(df.columns.str.startswith(('H_', 'V_')) & (~df.columns.isin(y_col)))]

print(df.shape)

(454, 134)
0/454
300/454
(374, 67)


In [72]:
df.to_csv(f'./baseballData/PREDICTVERSIONCombinedSlidingWindow{minimum_window}.csv')

In [73]:
df.tail()

,Date,Home_Team,Home_Wins,Home_Losses,Visitor_Team,Visitor_Wins,Visitor_Losses,Final_Score,Visitor_Final_Score,Home_Final_Score,Winning_Team,Home/Victory_Win,Game_Time,Winning_Pitcher,Visiting_Pitcher,Winning_Pitcher_Team,Losing_Pitcher_Team,Game_Time_(Duration),Attendance,Field_Conditions,Temperature,Wind_Direction,Cloud_Coverage,HP_Umpire,1B_Umpire,2B_Umpire,3B_Umpire,LF_Umpire,RF_Umpire,H_start_odds,V_start_odds,H_Won,D_AB,D_R,D_H,D_BB,D_PA,D_SO,D_BA,D_OBP,D_SLG,D_OPS,D_RBI,D_PIT,D_STR,D_WPA,D_aLI,D_P_H,D_P_R,D_ER,D_P_BB,D_P_SO,D_HR,D_ERA,D_CTCT,D_STS,D_STL,D_GB,D_FB,D_LD,D_GSC,D_P_WPA,D_P_aLI,D_start_odds,D_Final_Score,D_Final_Allowed,D_pythagorean
444,2025-05-01,CIN,NaN,NaN,SLN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,1.833333,0.0,4.440,3.451,3.422,0.073,4.510,1.316,0.056,0.043,0.111,0.154,3.452,13.517,8.679,0.158,0.032,-2.640,-1.133,-1.217,0.052,2.803,0.527,-1.295,3.251,4.570,-5.314,-4.171,-0.156,-2.628,13.015,0.188,-0.091,0.167,NaN,NaN,0.102
452,2025-05-01,ANA,NaN,NaN,DET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.250000,1.666667,0.0,-1.441,-2.430,-1.981,-1.180,-3.076,1.669,-0.053,-0.079,-0.084,-0.164,-2.417,-9.176,-5.219,-0.161,0.055,1.671,1.430,2.149,1.554,-2.187,0.473,2.370,4.610,-4.191,-0.831,-1.258,4.200,0.842,-15.196,-0.102,-0.159,0.583,NaN,NaN,-0.259
451,2025-05-01,TOR,NaN,NaN,BOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.952381,1.869565,0.0,-3.959,-5.155,-4.355,-1.343,-5.740,-1.082,-0.099,-0.109,-0.255,-0.364,-4.708,-34.650,-17.082,-0.216,0.236,3.736,1.888,2.119,-1.338,-1.839,1.065,2.215,4.067,-2.609,1.081,0.429,4.062,0.147,-15.422,-0.178,0.130,0.083,NaN,NaN,-0.102
446,2025-05-01,TBA,NaN,NaN,KCA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.625000,2.350000,0.0,3.364,0.480,0.603,0.736,3.429,3.110,-0.006,0.014,0.022,0.036,0.213,18.664,9.620,-0.041,0.310,-0.698,-0.038,-0.089,0.407,-1.976,-0.279,-0.199,1.280,-1.045,-0.555,1.368,-1.475,-1.678,-3.530,0.002,-0.038,-0.725,NaN,NaN,0.152
453,2025-05-01,SFN,NaN,NaN,COL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.363636,3.200000,0.0,-1.122,0.129,0.666,0.120,-1.290,-0.790,0.030,0.033,-0.027,0.006,-0.168,8.248,-0.275,0.195,0.174,-3.775,-3.050,-3.211,-1.309,3.028,-0.369,-3.028,-10.125,3.661,-0.971,-3.930,-3.759,-1.575,20.993,0.335,0.205,-1.836,NaN,NaN,0.348


### 3. Create an array of continuous values
Numpy array 'conts' containing stack of each continuous column

In [74]:

cont_cols = [
    # General
    
    # Random
    'D_AB',
    'D_R',
    'D_H',
    'D_BB',
    'D_PA',
    
    # Batting stats
    'D_SO',    # Strikeouts
    'D_BA',    # Batting Average
    'D_OBP',   # On-Base Percentage
    'D_SLG',   # Slugging Percentage
    'D_OPS',   # On-Base Plus Slugging
    'D_RBI',   # Runs Batted In
    'D_PIT',   # Pitches
    'D_STR',   # Strikes
    'D_WPA',   # Win Probability Added
    'D_aLI',   # Average Leverage Index

    # Pitching stats
    'D_P_H',   # Hits Allowed by Pitcher
    'D_P_R',   # Runs Allowed by Pitcher
    'D_ER',    # Earned Runs by Pitcher
    'D_P_BB',  # Walks by Pitcher
    'D_P_SO',  # Strikeouts by Pitcher
    'D_HR',    # Home Runs Allowed by Pitcher
    'D_ERA',   # Earned Run Average
    'D_CTCT',  # Contact Rate
    'D_STS',   # Swings and Misses (Strikes)
    'D_STL',   # Stolen Bases Against
    'D_GB',    # Ground Balls
    'D_FB',    # Fly Balls
    'D_LD',    # Line Drives
    'D_GSC',   # Game Score
    'D_P_WPA', # Pitcher Win Probability Added
    'D_P_aLI', # Pitcher Average Leverage Index

    # Odds
    'D_start_odds',

    # Calculated
    'D_pythagorean',
    
    # Remove later
    # 'D_Final_Score',
    # 'D_Final_Allowed'
]



df['Home_Team'] = df['Home_Team'].astype('category')
df['Visitor_Team'] = df['Visitor_Team'].astype('category')
df['Home_Team_Id'] = df['Home_Team'].cat.codes
df['Visitor_Team_Id'] = df['Visitor_Team'].cat.codes

# Retrieve original team name given a category code
home_cat_mapping = df['Home_Team'].cat.categories
home_team_from_id = lambda cat_code: home_cat_mapping[cat_code]
visitor_cat_mapping = df['Visitor_Team'].cat.categories
visitor_team_from_id = lambda cat_code: visitor_cat_mapping[cat_code]



drop_cols = [ 'D_BA', 'D_datediff', 'D_AB', 'D_P_WPA', 'D_P_H', 'D_ER', 'D_BA', 'D_ERA', 'D_SLG', 'D_H', 'D_STR' ]
y_col = ['H_Won', 'H_start_odds', 'V_start_odds', 'Home_Team_Id', 'Visitor_Team_Id']
y_col_teams = ['Home_Team', 'Visitor_Team']
cont_cols = [col for col in cont_cols if col not in drop_cols]

# Create cont_df and y_df from the filtered df
cont_df = df[cont_cols]

# Normalize cont_df
with open('MLB_scaler.pkl', 'rb') as f:
    standard_scaler = pickle.load(f)
cont_scaled = standard_scaler.transform(cont_df.values)
cont_df = pd.DataFrame(cont_scaled)

y_df = df[y_col]
y_df_team = df[y_col_teams]

In [75]:
test_sample = np.ones((2, 24))  # A single row with 24 features, all set to 1
print(standard_scaler.transform(test_sample))

[[ 0.55897418  0.81797202  0.34275438  0.52061139 20.72582661  7.55634593
   0.5738569   0.05184108  5.06941393  4.51944538  0.5427129   0.83762087
   0.54190607  1.58712444  0.20995798  0.3167198   0.30868288  0.49593567
   0.43779817  0.66507343  0.11682972  4.5293066   1.40366498  7.23391584]
 [ 0.55897418  0.81797202  0.34275438  0.52061139 20.72582661  7.55634593
   0.5738569   0.05184108  5.06941393  4.51944538  0.5427129   0.83762087
   0.54190607  1.58712444  0.20995798  0.3167198   0.30868288  0.49593567
   0.43779817  0.66507343  0.11682972  4.5293066   1.40366498  7.23391584]]


In [76]:
df.tail(15)

,Date,Home_Team,Home_Wins,Home_Losses,Visitor_Team,Visitor_Wins,Visitor_Losses,Final_Score,Visitor_Final_Score,Home_Final_Score,Winning_Team,Home/Victory_Win,Game_Time,Winning_Pitcher,Visiting_Pitcher,Winning_Pitcher_Team,Losing_Pitcher_Team,Game_Time_(Duration),Attendance,Field_Conditions,Temperature,Wind_Direction,Cloud_Coverage,HP_Umpire,1B_Umpire,2B_Umpire,3B_Umpire,LF_Umpire,RF_Umpire,H_start_odds,V_start_odds,H_Won,D_AB,D_R,D_H,D_BB,D_PA,D_SO,D_BA,D_OBP,D_SLG,D_OPS,D_RBI,D_PIT,D_STR,D_WPA,D_aLI,D_P_H,D_P_R,D_ER,D_P_BB,D_P_SO,D_HR,D_ERA,D_CTCT,D_STS,D_STL,D_GB,D_FB,D_LD,D_GSC,D_P_WPA,D_P_aLI,D_start_odds,D_Final_Score,D_Final_Allowed,D_pythagorean,Home_Team_Id,Visitor_Team_Id
272,2025-04-30,SEA,18.0,12.0,ANA,12.0,17.0,3-9,3.0,9.0,SEA,H,1:10 p.m.,Casey Legumina,Reid Detmers,SEA,ANA,2:41,"16,229",Unknown,56° F,6mph from Left to Right,Cloudy,Chris Segal,Alex MacKay,Tripp Gibson,Jim Wolf,NaN,NaN,1.714286,2.200000,1.0,3.188,5.383,4.517,3.869,8.217,-1.966,0.113,0.180,0.211,0.391,5.283,32.389,16.689,0.412,-0.127,-2.252,-0.948,-1.590,-1.085,1.550,0.658,-2.029,1.773,1.153,-3.071,-1.564,0.170,-1.640,20.861,0.156,0.122,-0.486,6.0,-6.0,0.180,23,0
260,2025-04-30,TEX,16.0,15.0,ATH,16.0,15.0,7-1,7.0,1.0,ATH,V,7:05 p.m.,Grant Holman,Robert Garcia,ATH,TEX,2:51,"19,803",NaN,74° F,0mph,In Dome,James Hoye,D.J. Reyburn,John Libka,Sean Barber,NaN,NaN,2.350000,1.625000,0.0,0.222,-1.307,-0.603,-1.469,-1.052,1.035,-0.019,-0.047,-0.040,-0.088,-1.133,-2.751,6.090,-0.370,0.224,-0.821,-1.358,-1.043,-0.191,-0.069,-0.675,-0.941,-4.178,-1.843,0.362,-0.494,-2.409,-0.063,0.468,-0.163,0.077,0.725,-6.0,6.0,-0.013,27,2
269,2025-04-30,SDN,19.0,11.0,SFN,19.0,12.0,3-5,3.0,5.0,SDN,H,1:10 p.m.,Michael King,Landen Roupp,SDN,SFN,2:34,"37,698",Unknown,65° F,10mph from Left to Right,Cloudy,Phil Cuzzi,Tony Randazzo,Jacob Metz,Dan Bellino,NaN,NaN,1.714286,2.200000,1.0,-0.865,-1.501,-1.605,-1.165,-1.799,0.556,-0.047,-0.066,-0.026,-0.092,-1.175,-19.090,-3.756,-0.430,-0.069,-0.059,0.277,0.288,0.696,-0.285,0.587,0.410,0.704,-1.949,1.710,-2.128,2.699,-0.565,-3.770,-0.110,-0.204,-0.486,2.0,-2.0,0.005,22,24
267,2025-04-30,CLE,17.0,13.0,MIN,13.0,18.0,2-4,2.0,4.0,CLE,H,6:10 p.m.,Cade Smith,Pablo LÃ³pez,CLE,MIN,2:24,"15,523",Unknown,49° F,9mph in from Centerfield,Cloudy,Malachi Moore,Andy Fletcher,Jansen Visconti,Ben May,NaN,NaN,1.952381,1.869565,1.0,-1.281,-3.674,-3.191,-1.217,-2.952,-0.362,-0.075,-0.093,-0.138,-0.231,-3.859,-28.124,-9.444,-0.223,-0.051,4.962,5.457,5.281,2.317,-0.230,0.909,5.185,12.117,-2.181,9.587,3.965,1.270,4.312,-21.743,-0.242,0.048,0.083,2.0,-2.0,-0.133,9,17
450,2025-05-01,PHI,NaN,NaN,WAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.571429,2.450000,0.0,-1.479,0.082,-1.239,1.054,0.158,-1.437,-0.024,-0.003,-0.092,-0.095,-0.276,1.243,-3.276,0.016,-0.017,-1.746,-3.326,-4.396,-0.562,1.960,-0.573,-4.429,2.999,1.535,-0.243,-1.506,-0.622,0.419,9.441,0.293,0.225,-0.879,NaN,NaN,0.092,20,29
449,2025-05-01,TEX,NaN,NaN,ATH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.769231,2.100000,0.0,1.916,2.270,2.049,0.649,2.962,0.276,0.039,0.034,0.025,0.059,2.401,16.440,12.819,-0.153,0.243,-3.116,-4.269,-4.033,-1.894,0.449,-0.256,-4.426,-10.135,0.618,-2.979,-1.121,-3.307,-2.048,9.603,0.003,-0.017,-0.331,NaN,NaN,0.090,27,2
448,2025-05-01,CHA,NaN,NaN,MIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.400000,1.645161,0.0,-1.173,-1.915,-1.894,-1.183,-2.620,1.133,-0.049,-0.065,0.020,-0.045,-1.639,-11.573,-0.959,-0.205,0.025,0.395,1.467,0.834,1.009,0.718,0.133,0.763,-6.987,1.708,-3.581,0.023,-1.059,-0.369,-0.183,-0.097,0.229,0.755,NaN,NaN,-0.191,6,16
447,2025-05-01,NYN,NaN,NaN,ARI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.606061,2.400000,0.0,1.077,3.519,3.264,0.422,2.220,-0.684,0.074,0.073,0.078,0.151,3.707,1.757,-0.297,0.236,-0.244,-0.459,-2.550,-2.121,-1.092,1.170,-0.767,-2.1

In [77]:
conts = np.stack([cont_df[col].values for col in list(cont_df.columns)], 1)
y_col = np.stack([y_df[col].values for col in list(y_df.columns)], 1)
#y_col_team = np.stack([y_df_team[col].values for col in y_col_teams], 1)


print(conts.shape)
print(y_col.shape)

(374, 24)
(374, 5)


In [78]:
conts[-15:,:]
# cont_df.tail()

array([[ 2.99553457,  3.18074215,  2.94914796, -1.05911663,  3.73767817,
         2.96061038,  3.0219137 ,  2.40527247,  2.09064183, -0.56745759,
        -0.53037009, -0.90522943,  0.83660267,  1.04244675,  0.35052084,
         0.36415672, -0.90544269, -0.73121715,  0.08819794, -1.07637538,
         2.16161129,  0.55525987, -0.38211469,  1.30213651],
       [-0.7235138 , -1.21537738, -0.39831803,  0.53925282, -0.96513854,
        -0.65409787, -0.6453131 , -0.22939506, -1.87092242,  1.01683961,
        -0.75622431, -0.15793534, -0.03087697, -1.08052211, -0.73161315,
        -0.56473883,  0.11840725, -0.21910579, -0.99809   , -0.03612357,
         0.06205787,  0.35157866,  1.07318758, -0.0940018 ],
       [-0.83136064, -0.96501763, -0.66809439,  0.28413154, -1.35876637,
        -0.68428333, -0.66931927, -1.45443299, -2.17487875, -0.30566489,
         0.14443826,  0.58350745, -0.14661236,  0.92937039,  0.15613313,
        -0.59760363,  0.52043163, -1.00115248,  1.15342563, -0.3672627 ,
  

### 4. Get only the current date games, load model, make prediction

In [79]:
conts_current_date = conts[len(conts) - len(current_date_games):]
y_col_current_date = y_col[len(conts) - len(current_date_games):]
#y_col_current_date_teams = y_col_team[len(conts) - len(current_date_games):]

print(conts_current_date.shape)
print(type(conts_current_date))

# conts_current_date = torch.tensor(conts_current_date, dtype=torch.float32)
# y_col_current_date = torch.tensor(y_col_current_date, dtype=torch.float32)
conts_current_date = torch.FloatTensor(conts_current_date)
y_col_current_date = torch.FloatTensor(y_col_current_date)

print(type(conts_current_date))

(11, 24)
<class 'numpy.ndarray'>
<class 'torch.Tensor'>


In [80]:
y_col_current_date_teams

array([['PHI', 'WAS'],
       ['TEX', 'ATH'],
       ['CHA', 'MIL'],
       ['NYN', 'ARI'],
       ['PIT', 'CHN'],
       ['CLE', 'MIN'],
       ['CIN', 'SLN'],
       ['ANA', 'DET'],
       ['TOR', 'BOS'],
       ['TBA', 'KCA'],
       ['SFN', 'COL']], dtype=object)

In [81]:
class TabularModelUpdated(nn.Module, BaseEstimator, RegressorMixin):
    def __init__(self, n_cont, out_sz, layer_shape, p=0.5, optimizer_class=torch.optim.Adam,
                 lr=0.001, confidence_threshold=0.1, batch_size=1000):
        super().__init__()
        # Model architecture params
        self.layer_shape = layer_shape
        self.batch_size = batch_size
        self.n_cont = n_cont
        self.out_sz = out_sz
        self.p = p
        self.lr = lr

        # Training params
        # self.criterion = criterion
        self.confidence_threshold = confidence_threshold
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Variable that holds the list of layers
        layerlist = []
        n_in = n_cont # no embed again
        # Iterate through the passed in "layers" parameter (ie, [200,100]) to build a list of layers
        for i, width in enumerate(self.layer_shape):
            # First layer gets special treatment
            if i == 0:
                layerlist.extend([
                    nn.Linear(n_in, width),
                    nn.Mish(),  # Mish instead of ReLU
                    nn.BatchNorm1d(width),
                    nn.Dropout(p/2)  # Less dropout in earlier layers
                ])
            else:
                layerlist.extend([
                    nn.Linear(n_in, width),
                    nn.Mish(),
                    nn.BatchNorm1d(width),
                    nn.Dropout(p)
                ])
            n_in = width
        # layerlist.append(nn.Linear(layers[-1], out_sz))
        
        # Final layer
        layerlist.extend([nn.Linear(self.layer_shape[-1], out_sz)])
        
        # Convert the list of layers into an attribute
        self.layers = nn.Sequential(*layerlist).to(self.device)
        
        # Initialize the optimizer & scheduler
        self.optimizer = optimizer_class(self.parameters(), lr=self.lr)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, mode='min',
                                                                    factor=0.1, patience=10,
                                                                    threshold=0.0001,threshold_mode='rel',
                                                                    cooldown=0, min_lr=0, eps=1e-08)
        
    def forward(self, x_cont):
        # x_cont = self.bn_cont(x_cont)  # Normalize the incoming continuous data
        x = self.layers(x_cont)        # Set up model layers
        return torch.clamp(x, -1, 1)
    
    def fit(self, X, y):
        """
        For sklearn pipeline
        """
        # Convert X,y to torch.tensor if needed
        if not isinstance(X, torch.Tensor):
            X = torch.FloatTensor(X).to(self.device)
        if not isinstance(y, torch.Tensor):
            y = torch.FloatTensor(y).to(self.device)
        
        # Initialize optimizer
        # optimizer = self.optimizer_class(self.parameters(), lr=self.lr)
        # Verify optimizer exists
        if not hasattr(self, 'optimizer'):
            raise ValueError("Optimizer not initialized. Set self.optimizer in __init__.")

        dataset = TensorDataset(X, y)
        dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=False)
        
        # Training loop
        running_loss = []    # This is only for training over final dataset
        self.train()
        for X_batch, y_batch in dataloader:
            self.optimizer.zero_grad()
            y_pred = self.forward(X_batch)[:,0]
            #loss = self.criterion(y_pred, y_batch)
            loss = nfl_custom_criterion(y_pred, y_batch)
            running_loss.append(loss)
            loss.backward()
            self.optimizer.step()
        # return self  # TODO: This being replaced w/ loss might break things
        return torch.mean(torch.stack(running_loss))

    def step_lr(self, val_loss):
        """
        Since CV is run outside of this class, we need to update the local self.optimizer variable
        after each CV fold.
        """
        self.scheduler.step(val_loss)
    
    def predict(self, X):
        self.eval()
        if not isinstance(X, torch.Tensor):
            X = torch.FloatTensor(X).to(self.device)
        with torch.no_grad():
            y_pred = self(X).squeeze()
        return y_pred
        
    def score(self, X, y):
        """
        12/5 - this isn't called at all if 'scoring' is defined
        """
        self.eval()
    
        if not isinstance(X, torch.Tensor):
            X = torch.FloatTensor(X)
        if not isinstance(y, torch.Tensor):
            y = torch.FloatTensor(y)
    
        dataset = TensorDataset(X, y)
        dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=False)
    
        total_loss = 0.0
        count = 0
    
        with torch.no_grad():
            for X_batch, y_batch in dataloader:
                y_pred = self.forward(X_batch)[:, 0]  # [batch_size]
                batch_loss = nfl_custom_criterion(y_pred, y_batch)  # scalar
                total_loss += batch_loss.item() * len(X_batch)
                count += len(X_batch)
    
        return total_loss / count

### Make predicitons

In [82]:
conts_current_date = torch.FloatTensor(conts_current_date)

# Model class must be defined somewhere
model = torch.load("MLB_Model.pt", weights_only=False)
model.eval()
x = model.predict(conts_current_date)

In [84]:
conts_current_date.shape

torch.Size([11, 24])

In [87]:
conts_current_date = torch.FloatTensor(conts_current_date)

# Model class must be defined somewhere
model = torch.load("MLB_Model.pt", weights_only=False)
model.eval()
x = model.predict(conts_current_date)

preds = x
max_bet_pct = 0.1

# Part of the loss function (cant call it directly since it expects h_won for y col)
h_start_odds = y_col_current_date[:,1]
v_start_odds = y_col_current_date[:,2]

y_decimal_odds = torch.where(x > 0, h_start_odds, v_start_odds) # Decimal odds for model's predicted outcome
x = torch.where(x > 0, (x + 1) / 2, (1 - x) / 2)
kelly_criterion = x - ((1 - x) / (y_decimal_odds - 1))
bet_multiplier = torch.clamp(kelly_criterion, min=0)   # Kelly results that are negative are ignored
bet_multiplier_numpy = bet_multiplier.numpy()
print(f"num bets {bet_multiplier_numpy.shape[0]}")
max_bet_pct = min((1.0 / bet_multiplier_numpy.shape[0]), 0.1)
bet_multiplier = bet_multiplier*max_bet_pct            # Scale down the bets to the maximum allowed percentage per bet



for i in range(bet_multiplier_numpy.shape[0]):
    # pred_team = 0 if preds[i] > 0 else 1
    # other_team = abs(pred_team - 1)
    #print(home_team_from_id(int(y_col_current_date[i][3])))
    #print(visitor_team_from_id(int(y_col_current_date[i][4])))
    pred_team = 3 if preds[i] > 0 else 4
    other_team = 3 if pred_team == 4 else 4
    pred_team = home_team_from_id(int(y_col_current_date[i][3])) # mlb_utils.get_team_name(y_col_current_date_teams[i][pred_team])
    other_team = visitor_team_from_id(int(y_col_current_date[i][4])) # mlb_utils.get_team_name(y_col_current_date_teams[i][other_team])
    # print(f"{bet_multiplier_numpy[i]} \n\tfor {pred_team} \n\tagainst {other_team}\n\todds {y_decimal_odds[i]}")
    print(f"{iso_date},{bet_multiplier[i]:.{4}},{mlb_utils.get_team_name(pred_team)},{mlb_utils.get_team_name(other_team)},{y_decimal_odds[i]:.{6}}")


num bets 11
2025-05-01,0.09091,Philadelphia Phillies,Washington Nationals,1.57143
2025-05-01,0.09091,Texas Rangers,Athletics,1.76923
2025-05-01,0.09091,Chicago White Sox,Milwaukee Brewers,2.4
2025-05-01,0.09091,New York Mets,Arizona Diamondbacks,1.60606
2025-05-01,0.09091,Pittsburgh Pirates,Chicago Cubs,2.05
2025-05-01,0.09091,Cleveland Guardians,Minnesota Twins,2.1
2025-05-01,0.09091,Cincinnati Reds,St. Louis Cardinals,1.83333
2025-05-01,0.09091,Los Angeles Angels,Detroit Tigers,2.25
2025-05-01,0.09091,Toronto Blue Jays,Boston Red Sox,1.86957
2025-05-01,0.09091,Tampa Bay Rays,Kansas City Royals,1.625
2025-05-01,0.09091,San Francisco Giants,Colorado Rockies,1.36364


https://www.espn.com/mlb/scoreboard/_/date/20250503

In [60]:
performance_tensor = nfl_custom_criterion(preds, y_col_current_date, 0.5, 0.05, True).numpy()
print(performance_tensor)

[0.95      0.95      0.95      0.95      1.1025    0.95      1.0916667
 0.95      1.0934782 0.95      0.95     ]


In [53]:
data = [[ 3.17708564e-01,  7.24910669e-01,  1.05421280e+00,
        -2.06404372e-01,  1.95027698e-01, -5.03887684e-02,
         5.51565469e-01,  2.61624793e-01,  3.47499079e-03,
         9.89757602e-01,  4.67795402e-01,  3.34409418e-01,
        -4.60598200e-01,  9.97853260e-01,  1.46313734e-01,
         5.71887030e-01, -9.89545712e-01, -6.21221229e-02,
         6.70303383e-01, -1.24260458e+00, -1.26821869e+00,
        -2.18728734e-01, -7.52249377e-01,  3.86049013e-05],
       [-1.36336801e+00, -1.70539070e+00, -4.12041594e-01,
         1.32869651e-01, -1.00657305e+00, -1.37854880e+00,
        -1.35406642e+00, -5.44295562e-01, -2.25086784e+00,
         2.64175981e+00, -2.87989948e-01, -4.96474612e-01,
        -2.84851861e-01, -7.63589775e-01, -3.63203898e-01,
        -6.18454463e-02, -5.87459311e-02, -1.40613957e-01,
         9.95704781e-02,  3.88024755e-01, -2.34657327e-01,
         8.76623556e-01, -9.85050027e-02, -6.94413610e-01],
       [ 1.95963931e-01,  6.86203735e-01,  2.64385616e-01,
        -3.27839972e-01,  9.20131595e-01,  1.83548509e-01,
         7.31569848e-02,  1.11575492e+00,  5.10068884e-01,
         7.73101575e-01, -7.17112971e-01, -6.10156934e-01,
         9.54481317e-01, -4.69086126e-02, -1.30695711e+00,
         4.81428092e-03,  6.88339442e-01, -1.03417648e+00,
        -1.06506161e+00, -6.08690831e-01,  9.13595470e-01,
        -1.82518740e-01, -2.20755165e+00,  2.40168585e+00],
       [ 2.19613374e+00,  3.09840561e-01,  1.69596995e+00,
         1.34136692e+00,  1.72810451e+00,  2.16069582e+00,
         2.20913370e+00,  1.61944577e+00,  1.36114663e+00,
        -1.63268306e+00,  2.85459432e-01,  3.19363229e-01,
         9.81807729e-01,  2.63653225e-01, -1.75846756e+00,
         4.88174815e-01,  3.48348494e-01, -8.38904116e-01,
        -6.48910977e-01, -2.68976386e-01,  3.15635078e-01,
        -1.09229482e+00, -1.64033227e+00,  6.87256942e-01],
       [ 1.01482167e+00,  1.42492970e+00,  9.21310920e-01,
        -1.47189115e+00,  1.45878020e+00,  1.25513217e+00,
         9.75674305e-01,  2.73321098e-01,  3.83420411e-01,
        -1.00528331e+00, -2.46996208e+00, -3.55207609e-01,
         1.01717132e+00, -1.41815857e+00, -9.76370727e-01,
         1.98282512e-01, -6.93395702e-01, -1.71667255e+00,
        -7.45788149e-01, -1.47018028e+00,  1.35588849e+00,
         1.07125227e+00, -1.18006739e+00,  1.79404016e+00],
       [-1.19159133e+00, -1.69797874e+00, -5.66612254e-01,
        -3.92818670e-01, -8.40835012e-01, -1.00123061e+00,
        -1.32834553e+00, -6.92673680e-01, -3.61272613e-01,
         2.63057178e-01, -7.04932937e-03,  2.99301642e-01,
        -4.34343411e-01,  4.72287074e-01, -1.00444091e-01,
         4.88484860e-01, -6.85343284e-01, -6.22572963e-01,
         6.89678817e-01,  6.94097575e-01,  2.79909828e-01,
        -1.50475227e-02, -9.50535815e-01, -6.51010346e-01],
       [-2.19445580e+00, -1.11160985e+00, -1.74431067e+00,
         2.74544518e-01, -1.37948362e+00, -1.63512516e+00,
        -2.33088852e+00, -1.39362720e+00, -8.17207117e-01,
        -3.19205895e-01,  2.76645609e-01,  5.52579169e-01,
        -1.72886523e+00,  4.26100955e-01, -1.47479608e+00,
        -1.27908307e+00, -1.02861485e+00,  8.09903010e-01,
        -7.94416876e-02, -1.83223225e-01, -1.22724275e+00,
         1.38844948e-01,  1.23908392e-03, -1.19355114e+00],
       [ 4.67248775e-01,  1.52293237e+00,  7.76130066e-01,
         3.44316727e-01,  1.08586963e+00, -1.10759679e-01,
         3.36653175e-01,  3.46123094e-01,  1.15344313e+00,
         3.44303188e-01, -2.87989948e-01,  5.17471393e-01,
        -3.22358701e-01,  3.03468845e-01, -1.35721187e-01,
        -4.52812498e-01, -1.80725140e-01,  1.01477975e-02,
        -3.54488615e-01, -1.03415844e+00, -2.22508683e-01,
        -7.93562373e-01, -9.85050027e-02, -3.90590766e-01],
       [-6.26785185e-01,  9.93388557e-01,  8.45470175e-01,
         4.91850328e-01,  7.07241728e-02, -3.44696956e-01,
        -6.67032964e-01,  3.64042433e-01, -1.17688878e+00,
         1.25606397e+00,  9.72386787e-01,  2.21769083e+00,
        -6.71172319e-01, -8.40035766e-01, -2.25011522e-03,
        -1.73639980e+00,  3.56102674e-01,  1.89490903e+00,
        -1.14846022e+00, -1.28482152e+00, -6.79421255e-01,
         1.04409477e+00,  1.91800798e+00, -1.39609970e+00],
       [ 2.02958766e-02,  7.31499084e-01, -3.84955614e-01,
         2.16638334e+00, -8.40835012e-01, -7.22015146e-01,
        -2.28676573e-02,  3.71090206e-01,  1.65585037e-01,
        -8.78900632e-01,  3.16702879e+00, -2.25196370e-02,
         6.79073944e-01,  2.49014270e+00,  5.72912008e-01,
         1.47535887e+00,  9.18280689e-01,  1.04729295e+00,
         6.40397821e-01,  6.44624598e-01, -2.48205357e+00,
        -2.10164837e+00, -3.82114695e-01, -4.62929538e-01],
       [ 7.14397406e-02, -3.07823292e-01,  3.02305988e-01,
        -1.11983443e+00, -5.30076199e-01, -4.57892413e-01,
         8.51600651e-02, -7.89925829e-02, -1.99162567e-01,
        -5.90025929e-01,  2.04987669e+00,  1.25473469e+00,
        -9.67818453e-02,  1.97413227e+00, -1.43358496e-01,
         8.29844672e-01,  1.51511629e-01, -1.32477608e-01,
         6.65670127e-01,  1.09251995e+00, -1.82860064e+00,
        -2.04280713e+00, -2.89581024e-01, -4.05058520e-01]]

tensor = torch.tensor(data, dtype=torch.float32)

print(model.predict(tensor))

tensor([ 1., -1.,  1.,  1.,  1., -1., -1., -1.,  1., -1.,  1.])


In [24]:
"""
Want: 
    An array > 1 elements for PCC formula
    A smaller array for kelly_criterion
    


x: 1d array of predictions between -1 and 1 where negative number means visitor predicted to win
y: ['H_Won', 'H_start_odds', 'V_start_odds']
pearson_multiplier: constant to multiply the pearson correlation coefficient's result by
max_bet_pct: max percentage allowed per bet.
return_res_array: Skip the torch.cumprod part and ignore PCC penalization
"""
def nfl_custom_criterion(x, y, pearson_multiplier=0.5, max_bet_pct=0.1, return_res_array=False):
    # ------------------------------------------------
    # Preliminary calculations
    # ------------------------------------------------
    # acct_value = 100 # Preset account value
    batch_size = len(x)
    h_start_odds = y[:,1]
    v_start_odds = y[:,2]
    h_won = y[:,0]
    y_decimal_odds = torch.where(x > 0, h_start_odds, v_start_odds) # Decimal odds for model's predicted outcome
    y_prob = 1 / y_decimal_odds                  # Implied Probability (regardless of correct prediction)
    x_H_Won = torch.round(torch.sigmoid(20 * x)) # H_won for predicted bets (Converts model's -1 to 1 range to 0 to 1)
                                                 # Sigmoid so that it's differentiable. The 20 is arbitrarily large number
    y_incorrect_prediction = torch.abs((x_H_Won - h_won))        # 1 if wrong bet, otherwise 0. Used to reset kelly when wrong
    y_incorrect_prediction_mult_two = 2 * y_incorrect_prediction   # 2 if wrong bet, 0 if correct

    #x = torch.abs(x)         # OLD VERSION
    x_prob = torch.where(x > 0, (x + 1) / 2, (1 - x) / 2)
    x = x_prob                # x is now the implied probability(?) of your prediction.
                              # It's a number between 0 and 1 (formerly -1 and 1) representing the model's predicted probability of a win.
                              # This now only shows the probability. Not whether it was correct & not for which side (home vs visitor)

    
    # ------------------------------------------------
    # 1. Calculate the Pearson Correlation Coefficient
    #    Currently includes cases where predicted wrong
    #    ^ This is filtered out after step 2
    # ------------------------------------------------
    n = x.size(0)
    sum_x = torch.sum(x)
    sum_x_squared = torch.sum(x**2)
    sum_y = torch.sum(y_prob)
    sum_y_squared = torch.sum(y_prob**2)
    sum_pow_x = torch.sum(x**2)
    sum_pow_y = torch.sum(y_prob**2)
    x_mul_y = torch.mul(x, y_prob)
    sum_x_mul_y = torch.sum(x_mul_y)

    
    # PCC Formula (eps to avoid NaN)
    eps = 1e-8
    pcc_numerator = n * sum_x_mul_y - sum_x * sum_y
    pcc_denominator_one = torch.sqrt(n * sum_pow_x - sum_x_squared + eps)
    pcc_denominator_two = torch.sqrt(n * sum_pow_y - sum_y_squared + eps)
    pcc = pcc_numerator / (pcc_denominator_one * pcc_denominator_two + eps)
    pcc = pearson_multiplier * torch.abs(pcc)

    
    # ------------------------------------------------
    # 2. Calculate the kelly criterion
    #    Entirely wrong predictions are negated and kept in "incorrect_bets" (pcc not applied to wrong predictions)
    #    Correct predictions are kept in "correct_bets". Pcc is applied to this & stored in pcc_adjusted_correct_bets
    #    Possible issue: This always bets max_bet_pct
    #    The result is cumulatively calculated. i.e. The sum of the previous values are used to calculate the next one
    # ------------------------------------------------
    # kelly_criterion = x - ((1 - x) / y_decimal_odds)  # OLD VERSION
    kelly_criterion = x - ((1 - x) / (y_decimal_odds - 1))
    bet_multiplier = torch.clamp(kelly_criterion, min=0)   # Kelly results that are negative are ignored
    bet_multiplier = bet_multiplier*max_bet_pct            # Scale down the bets to the maximum allowed percentage per bet


    # 4/5/25 adjustment of kelly
    #    Want to use cumprod. Cumsum does nothing and is the same as torch.sum in this scenario?
    #    Basically start with max_bet_pct and return as if you made the bets sequentially
    correct_bet_multiplier = bet_multiplier - (bet_multiplier * y_incorrect_prediction)          # Correct bets after kelly. Bet multiplier or 0
    if not return_res_array:
        correct_bet_multiplier = correct_bet_multiplier * (1 - pcc)                              # "correct_bet_multiplier" penalized by pcc
    assert torch.all(correct_bet_multiplier <= max_bet_pct), "Correct bet mult. can't exceed max bet pct"

    correct_bet_multiplier = correct_bet_multiplier * y_decimal_odds                             # Bet multiplier taking market odds into account
    incorrect_bet_multiplier = bet_multiplier - (bet_multiplier * y_incorrect_prediction_mult_two) # Negative numbers are incorrect bets
    incorrect_bet_multiplier = torch.clamp(incorrect_bet_multiplier, max=0)                      # Restrict to 0 or negative
    combined_bet_multiplier = correct_bet_multiplier + incorrect_bet_multiplier                  # Combine correct & incorrect bet multipliers
    combined_bet_multiplier = combined_bet_multiplier + 1                                        # Converts to format friendly to cumprod
                                                                                                 # Ex: loss=-0.3, profit=0.3 --> loss=0.7, profit=1.3

    assert torch.all((x >= 0) & (x <= 1)), "Probabilities must be between 0 and 1"
    assert torch.all(y_decimal_odds > 1), "Decimal odds must be greater than 1"
    assert torch.all(kelly_criterion <= 1), "Kelly Criterion cannot be greater than 1"
    assert torch.all(incorrect_bet_multiplier >= -max_bet_pct), "Incorrect bet mult. can't exceed max bet pct"
    
    # ------------------------------------------------
    # Combine & Return
    #     Negate everything for Adam & optuna
    # ------------------------------------------------
    if return_res_array:
        return combined_bet_multiplier
        
    # Prepend max_bet_pct to the tensor before torch.cumprod
    res = torch.sum(combined_bet_multiplier) / batch_size
    # print(res)
    return -res